In [ ]:
import json

In [ ]:
# Conversation (thread) properties:
#
# participants: object array:
#        name: string
#
# messages: object array
#        sender_name: string - coincides with one of the participants
#        timestamp_ms: integer - unix time in ms
#        content: string
#        type: string - we'll focus on Generic (basic text)
#
# title: string
#
# is_still_participant: boolean
#
# thread_type: string
#
# thread_path: string

In [ ]:
file = 'message_1.json'

In [ ]:
with open(file) as jsonfile:
    data = json.load(jsonfile)
    participants = [p['name'] for p in data['participants']]
    messages_all = data['messages']
    messages_generic = [message for message in messages_all if message['type'] == 'Generic' and 'content' in message]
    messages = messages_generic
    
    print(participants)
    print(messages[0])

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

# get word frequency for given participant
def word_freq(participant, messages):
    messages_of_participant = []
    for message in messages:
        if message['sender_name'] == participant:
            messages_of_participant.append(message['content'])
    
    content = []
    for message in messages_of_participant:
        content.extend(list(map(str.lower, message.split())))
        
    freqdict = wordListToFreqDict(content)
    sortdict = sortFreqDict(freqdict)
        
    for entry in sortdict:
        print(entry)

In [ ]:
word_freq(participants[0], messages)

In [ ]:
# questions for a participant
def question_count(participant, messages):
    messages_of_participant = []
    for message in messages:
        if message['sender_name'] == participant:
            messages_of_participant.append(message['content'])
    
    count = 0
    
    for message in messages_of_participant:
        if '?' in message:
            count += 1
            
    return count

In [ ]:
for p in participants:
    print(p, 'question count:', question_count(p, messages))

In [ ]:
# conversation starter
def conversation_starter(participants, messages, interval=3600):
    length = len(messages)
    interval *= 1000
    count = {}
    for p in participants:
        count[p] = 0
        
    # set the first conversation
    count[messages[length - 1]['sender_name']] = 1
    
    for i in range(length - 2, -1, -1):
        if messages[i]['timestamp_ms'] - messages[i + 1]['timestamp_ms'] > interval:
            count[messages[i]['sender_name']] += 1
            
    return count

In [ ]:
interval = 3600
convo = conversation_starter(participants, messages, interval)
print('Conversations started (1 hour for a new convo):')
for p in convo:
    print(p, convo[p])

In [ ]:
interval = 24 * 3600
convo = conversation_starter(participants, messages, interval)
print('Conversations started (1 day for a new convo):')
for p in convo:
    print(p, convo[p])

In [ ]:
interval = 48 * 3600
convo = conversation_starter(participants, messages, interval)
print('Conversations started (2 days for a new convo):')
for p in convo:
    print(p, convo[p])